In [ ]:
import json
import os
from pathlib import Path
import pandas as pd

from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
from pgvector.sqlalchemy import Vector
from sqlalchemy import Index, create_engine, select, text
from sqlalchemy.orm import Session


In [2]:

# Connect to the database based on environment variables
load_dotenv(".env", override=True)
POSTGRES_HOST = os.environ["POSTGRES_HOST"]
POSTGRES_USERNAME = os.environ["POSTGRES_USERNAME"]
POSTGRES_DATABASE = os.environ["POSTGRES_DATABASE"]

if POSTGRES_HOST.endswith(".database.azure.com"):
    print("Authenticating to Azure Database for PostgreSQL using Azure Identity...")
    azure_credential = DefaultAzureCredential()
    token = azure_credential.get_token("https://ossrdbms-aad.database.windows.net/.default")
    POSTGRES_PASSWORD = token.token
else:
    POSTGRES_PASSWORD = os.environ["POSTGRES_PASSWORD"]

DATABASE_URI = f"postgresql://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}/{POSTGRES_DATABASE}"
# Specify SSL mode if needed
if POSTGRES_SSL := os.environ.get("POSTGRES_SSL"):
    DATABASE_URI += f"?sslmode={POSTGRES_SSL}"

engine = create_engine(DATABASE_URI, echo=False)

Authenticating to Azure Database for PostgreSQL using Azure Identity...


In [4]:
# Run query and print results
with Session(engine) as session:
    most_similars = session.execute(text(query)) #.scalars()
    print(f"Most similar recipes to 'chicken and peanut':")
    print(f"--------------------------------------------------")
    for Recipe in most_similars:
        print(f"INPUT: {Recipe[0]}: \nOUTPUT: {Recipe[1]} SCORE: ({Recipe[2]})")
        print(f"--------------------------------------------------")

Most similar recipes to 'chicken and peanut':
--------------------------------------------------
INPUT: chicken and peanut: 
OUTPUT: Sweet and Sour Chicken III SCORE: (0.20339465)
--------------------------------------------------
INPUT: chicken and peanut: 
OUTPUT: Sticky Orange Chicken with Nectarines SCORE: (0.19649275)
--------------------------------------------------
INPUT: chicken and peanut: 
OUTPUT: Spinach Salad with Chicken, Avocado, and Goat Cheese SCORE: (0.21424025)
--------------------------------------------------
INPUT: chicken and peanut: 
OUTPUT: Southern Apricot Chicken SCORE: (0.2084128)
--------------------------------------------------
INPUT: chicken and peanut: 
OUTPUT: Peanut Butter Mango Smoothie SCORE: (0.19763698)
--------------------------------------------------
INPUT: chicken and peanut: 
OUTPUT: Peanut Butter Banana Smoothie SCORE: (0.19594538)
--------------------------------------------------
INPUT: chicken and peanut: 
OUTPUT: Papaya-Stuffed Chicken a